<a href="https://colab.research.google.com/github/BC-Chang/porescale_permeability_2d/blob/master/Permeability_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Permeability Estimation

In this activity, we will compute the single phase permeability of a 2D sample using lattice Boltzmann (LBM) simulations.

Before we get started, let's import some packages.

In [ ]:
#@title Import Packages
%%capture
!pip install porespy

import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import tifffile
import porespy as ps

from skimage.measure import regionprops,label
from skimage.morphology import binary_erosion, binary_dilation

import sys
import os
from tqdm import tqdm

try:
  os.chdir("./porescale_permeability_2d")
  # sys.path.append("./LBM_Workshop/")
except:
  !git clone https://github.com/BC-Chang/porescale_permeability_2d.git
  os.chdir("./porescale_permeability_2d")

from plotting_utils import plot_profile, plot_quiver, plot_streamlines
from lbm import run_lbm

# Import a timer
from time import perf_counter_ns, sleep

# Import ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from IPython.display import display, clear_output

import torch


## Defining the lattice Boltzmann method

On a basic level, LBM is a common simulation method for modeling how fluids move by tracking groups of particles on a grid. Instead of solving complex equations (Navier-Stokes) directly, LBM follows simple rules for how these particles collide and flow. We can then visualize and calculate a value for permeability the simulation results.

If you are interested in specifics, see the [D2Q9_LBM_main.ipynb notebook](https://github.com/BC-Chang/porescale_permeability_2d/blob/master/D2Q9_LBM_main.ipynb).

###  Read in the geometry
Before we get into the flow simulation, let's first read in our geometry. For this exercise, we assume a 2D binary image with:
- 0 indicating fluid space
- 1 indicating solid space

For this workshop, we select an image from the data folder. You can also load in your own image or create one yourself if you'd like.


In [ ]:
Nx = 150
Ny = 150
X, Y = np.meshgrid(range(Nx), range(Ny))
data = (X - Nx/4)**2 + (Y - Ny/2)**2 < (Ny/8)**2

plt.imshow(data, cmap='binary')
plt.colorbar()


Let's run our LBM simulation! This simulation does not fully converge in the default number of iterations, but it gets the point across. This should take about one minute to run.

In [ ]:
u_x, u_y, u = run_lbm(data)

In [ ]:
_ = plot_profile(u, cmap='jet')

In [ ]:
#@title Read in data
geom_options = os.listdir("./data/")

# Widget to read in from drop down, and plot.
data_dropdown = widgets.Dropdown(
    concise=False,
    options=geom_options,
    value='segmented_gambier.tif',
    description='Select a file to read in'
)

erosion_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Erosion steps'
)

# output = widgets.Output()
def read_in_and_plot(dropdown, erosion_slider):
  display(dropdown)
  data = tifffile.imread(os.path.join("./data/", dropdown))
  clear_output(wait=True)
  display(dropdown)
  display(erosion_slider)
  for i in range(max(1, erosion_slider)):
    data = binary_erosion(data)
  plt.imshow(data, cmap='binary')
  plt.colorbar()
  plt.show()

  return data

widget = widgets.interactive(read_in_and_plot, dropdown=geom_options, erosion_slider=erosion_slider)
display(widget)


In [ ]:
data = widget.result

u_x, u_y, u = run_lbm(data)

In [ ]:
# from plotting_utils import plot_quiver
profile_fig = plot_profile(u, cmap='jet')

## Calculate Permeability

Once we have the velocity field, we can compute the absolute permeability using Darcy's law:

$$k = \frac{\bar{u} \mu L}{\Delta P}$$

Since we used a force to drive flow (instead of pressure), we can equivalently compute permeability as

$$k = \frac{\bar{u} \mu}{F}$$


Keep in mind that the calculated permeability will be in lattice units ($lu^2$). To convert to the true permeability, we would need to know the physical size of our grid.


In [ ]:
def permeability(ux, tau=1.0, F=0.00001):
  u_mean = torch.nanmean(ux[ux != 0])
  print(f"Average Velocity = {u_mean} lu/ts")

  mu = (tau - 0.5) / 3
  permeability = u_mean * mu / F

  print(f"Permeability = {permeability} lu^2")
  return

# Calculate the average velocity for our image
permeability(u_x)

## Porosimetry

Porosimetry is a method used to measure the numbers and sizes of pores. It helps us understand how easily fluids (or small beads) can pass through the material.

### Traditional Drainage Curve

In the following cell we will run a porosimetry simulation and plot the *drainage curve*.

We want to measure how *capillary pressure*, which is inversely related to pore radius, varies with *saturation* (how much of one fluid is present versus the other). In other words, we are measuring how much pressure must be applied to push a certain amount of fluid into the porous material.

**The key idea:**

To push more fluid into the sample (i.e., to increase saturation), we need to overcome capillary forces in progressively smaller pores — which means applying higher pressure.


In [ ]:
data = widget.result
# Invert the data. Pores should be labeled 1, solids 0
data = (data == 0).astype(np.uint8)
# Define inlet
inlet = np.zeros_like(data)
# Inlet is on the left boundary of the image
inlet[:, 0] = True

# Run porosimetry from PoreSpy
mip = ps.filters.porosimetry(im=data, inlets=inlet)
pore_radii = ps.metrics.pore_size_distribution(mip, log=False)
saturation = 1 - pore_radii.cdf

fig, ax = plt.subplots(1, 2, figsize=[8, 4])
ax[0].imshow(mip)
ax[1].plot(saturation, 1/pore_radii.R, 'bo-')
ax[1].set_xlabel('Saturation');
ax[1].set_ylabel('1/Pore Radius [voxels]')
ax[1].set_xlim([-0.10, 1])

plt.tight_layout()

**Another way to think about this:**

Imagine rolling a ball of radius $r$ into the porous medium. The ball can only enter regions where the pores are large enough to fit it. As we decrease the radius of the ball, it can reach deeper into the structure. This is similar to how increasing capillary pressure allows fluid to invade smaller pores.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[8, 4])
ax[0].imshow(mip)
ax[1].plot(pore_radii.R, 1-saturation, 'bo-')
ax[1].set_ylabel('Invaded Saturation');
ax[1].set_xlabel('Pore Radius [voxels]')

plt.tight_layout()